Maps FBRef player names to FPL APIL player codes

In [ ]:
import pandas as pd
import rapidfuzz

In [ ]:
df_fbref = pd.read_csv("clean/fbref_general.csv", usecols=["team", "player", "pos", "born"])
df_fpl_api = pd.read_csv("clean/fpl_api_2425.csv", usecols=["birth_date", "code", "first_name", "second_name", "web_name", "position", "team_name"])
df_fpl_api["full_name"] = df_fpl_api["first_name"] + " " + df_fpl_api["second_name"]

In [ ]:
def map_player_names(fbref_row, df_fpl_api, score_cutoff: int = 80, ret_limit: int = 5) -> tuple[str, int]:
    matches = rapidfuzz.process.extract(
        query=fbref_row.loc["player"],
        choices=df_fpl_api["full_name"],
        scorer=rapidfuzz.fuzz.token_set_ratio,
        score_cutoff=score_cutoff,
        limit=ret_limit
    )
    if len(matches) == 1:
        choice, _, index = matches[0]
        return choice, df_fpl_api.iloc[index]["code"]
    else:
        for choice, _, index in matches:
            print(f"Potential match: FBRef={fbref_row.loc["player"]}, FPL={choice}")
            print(fbref_row)
            print(df_fpl_api.iloc[index])
            print("-"*80)
            confirmation = input("Is this a match? (y/n): ").strip().lower()
            if confirmation == "y":
                return choice, df_fpl_api.iloc[index]["code"]
    return "", -1

In [ ]:
mapped: dict[str, int] = {}
failed_matches = []

for idx, row in df_fbref.iterrows():
    player_name, code = map_player_names(row, df_fpl_api)
    if code != -1:
        mapped[player_name] = code
    else:
        failed_matches.append((idx, row.loc["player"]))

In [ ]:
len(failed_matches)

In [ ]:
df_translate = pd.DataFrame.from_dict({"fbref_name": list(mapped.keys()), "fpl_code": list(mapped.values())})
df_translate.to_csv("clean/fbref_to_fpl_api_2425.csv", index=False)